In [2]:
import pandas as pd 
import numpy as np
import csv
import advance_functions as advance
import stumps
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc

In [3]:
### KY data
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
KY_data = KY_data.drop(['PersonID', 'screening_date','ADE', 'Treatment','p_assault','p_pending_charge','fta_risk_score_raw',
                        'nca_risk_score_raw', 'pvf_risk_score_raw', 'fta_calc', 'nca_calc', 'pvf_calc', 
                        'recid_traffic_two_year', 'recid_traffic_six_month'], axis=1)

KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_two_year'].values

In [4]:
### FL data
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv")
FL_data = FL_data.drop(['person_id', 'screening_date','p_age_first_offense', 'p_famviol_arrest', 'p_juv_fel_count', 'p_domestic',
                        'current_violent20', 'pending_charge', 'total_convictions', 'years_since_last_crime'], axis=1)

FL_data.columns = ['Gender', 'age_at_current_charge', 'p_charges', 'p_jail30', 'p_prison', 
                   'p_probation', 'p_felprop_viol','p_murder', 
                   'p_felassult', 'p_misdeassult', 'p_sex_offense', 
                   'p_weapon', 'p_fta_two_year', 'p_fta_two_year_plus', 
                   'current_violence', 'p_felony', 'p_misdemeanor', 
                   'p_violence', 'p_arrest', 'p_property', 'p_traffic', 'p_drug', 
                   'p_dui', 'p_stalking', 'p_voyeurism', 'p_fraud', 'p_stealing', 
                   'p_trespass', 'six_month', 'one_year', 'three_year', 'five_year', 
                   'recid_two_year', 'recid_six_month', 'recid_drug_two_year', 'recid_property_two_year',
                   'recid_M_two_year', 'recid_F_two_year', 'recid_violent_two_year', 'recid_drug_six_month',
                   'recid_property_six_month', 'recid_M_six_month', 'recid_F_six_month', 'recid_violent_six_month']

reorder = ['Gender', 'age_at_current_charge', 'p_arrest', 'p_charges',
       'p_violence', 'p_felony', 'p_misdemeanor', 'p_property', 'p_murder',
       'p_sex_offense', 'p_weapon', 'p_felprop_viol', 'p_felassult',
       'p_misdeassult', 'p_traffic', 'p_drug', 'p_dui', 'p_stalking',
       'p_voyeurism', 'p_fraud', 'p_stealing', 'p_trespass', 'p_prison',
       'p_jail30', 'p_fta_two_year', 'p_fta_two_year_plus', 'p_probation',
       'six_month', 'one_year', 'three_year', 'five_year', 'current_violence',
       'recid_two_year', 'recid_drug_two_year', 'recid_violence_two_year',
       'recid_F_two_year', 'recid_M_two_year', 'recid_property_two_year',
       'recid_six_month', 'recid_drug_six_month', 'recid_violence_six_month',
       'recid_F_six_month', 'recid_M_six_month', 'recid_property_six_month']

FL_data = FL_data.reindex(columns=reorder)
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_two_year'].values

### EBM

In [ ]:
#### GAM
estimators = [80]
depth = [2]
learning_rate = [0.3]
ebm_auc, ebm_std, ebm_auc_diff, ebm_param, ebm_FL_score = advance.EBM(KY_X,KY_Y, FL_X, FL_Y,learning_rate,depth,estimators,816)

### Lasso Stumps

In [ ]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv")
X_stumps, Y_stumps = data.loc[:,:'current_violence>=1'], data['recid_two_year'].values
Y_stumps[Y_stumps == -1] = 0
cols = data.columns[:-14]

In [ ]:
### stumps model
alpha = [0.001]
Stump = stumps.stump_features(X_stumps, Y_stumps, cols, alpha, 816)

### Results

In [9]:
results = [["AdaBoost", ada_auc, ada_auc_diff, ada_param],
           ["EBM", ebm_auc, ebm_auc_diff, ebm_param], 
           ['Lasso Stumps', round(np.mean(Stump['test_auc']),3), round(np.std(Stump['test_auc']),3)], 
           ['Arnold PSA Raw', round(np.mean(raw_auc), 3), round(np.std(raw_auc), 3)], 
           ['Arnold PSA', round(np.mean(calc_auc), 3), round(np.std(calc_auc), 3)]]

In [10]:
results

[['AdaBoost',
  0.7378145162357288,
  0.0016431431438883592,
  {'learning_rate': 1, 'n_estimators': 90}],
 ['EBM',
  0.7371276615810134,
  0.0033369533038599686,
  {'learning_rate': 0.3, 'max_tree_splits': 2, 'n_estimators': 80}]]

In [7]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY Results/Models/Two Year/"
results = [["", "AdaBoost", "std", "EBM", "std", "Lasso Stumps", "std", "Arnold PSA Raw", "std", "Arnold PSA", "std"],
           ["Two Year Recidivism", round(ada_auc,3), round(ada_std, 3), round(ebm_auc,3), round(ebm_std, 3), round(np.mean(Stump['test_auc']),3), round(np.std(Stump['test_auc']),3), round(np.mean(raw_auc), 3), round(np.std(raw_auc), 3), round(np.mean(calc_auc), 3), round(np.std(calc_auc), 3)]]
with open(path + 'Two Year Models Summary.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)